In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5,7"
import numpy as np
import sys
import scipy.ndimage as nd
import json
import pickle
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from resnet import *
import torch.optim as optim
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import time
import math
from utils import AverageMeter
import cv2
from PIL import Image
import torchvision.transforms as transforms
import torch.nn.functional as F
from dr_model import DRModel

In [9]:
class BinClsDataSet(torch.utils.data.Dataset):
    def __init__(self, config_file):
        self.images_list = []
        self.labels_list = []
        with open(config_file, 'r') as f:
            for line in f.readlines():
                line = line.strip()
                if line is None or len(line) == 0:
                    continue
                ss = line.split('\t')
                if len(ss) != 2:
                    continue
                if not os.path.isfile(ss[0]):
                    continue
                self.images_list.append(ss[0])
                self.labels_list.append(ss[1])
        self.transform = transforms.Compose([
#             transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
#             transforms.Normalize(mean=[0], std=[255])
        ])
    def __getitem__(self, item):
        image = self.transform(Image.open(self.images_list[item]))
        return image, int(self.labels_list[item]), self.images_list[item]
    def __len__(self):
        return len(self.images_list)

In [10]:
ds = BinClsDataSet(config_file='/data/zhangwd/data/examples/dr/train_label.txt')
dataloader = DataLoader(ds, batch_size=2, 
                                     shuffle=True, num_workers=2, 
                                     pin_memory=True)
for i, (images, labels, files) in enumerate(dataloader):
    print(images.shape)
    print(labels)
    break

torch.Size([2, 1, 1024, 1024])
tensor([1, 0])


In [11]:
files

['/data/zhangwd/data/examples/dr/total_pos_train/thread_4_161.jpg',
 '/data/zhangwd/data/examples/dr/total_neg_train/thread_7_140.jpg']

In [13]:
images.numpy()[0].max()

1.0